Author: Mohamed-Amine Baazizi
Affiliation: LIP6 - Faculté des Sciences - Sorbonne Université
Email: mohamed-amine.baazizi@lip6.fr

# Interrogation de données structurées en Spark

Le but du TME est de formuler des requêtes SQL en utilisant l'API Dataframe de Spark, en Scala.
Pour la documentation à consulter, suivre ces liens:
* https://spark.apache.org/docs/latest/sql-programming-guide.html
* https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset


Ce TME  utilise le jeux de données Books qui renseigne sur des livres (books.csv), des utilsateurs (users.csv) et des notes réalisées par les utilsateurs (ratings.csv).

Le schéma de la base est  

* `Users (userid: Number, country: Text, age: Number)`
* `Books (bookid: Number, titlewords: Number, authorwords: Number, year: Number, publisher: Number)`
* `Ratings (userid: Number, bookid: Number, rating: Number)`

## Rappels de quelques fonctions

|Expression |Action|
|:-------------:|:-------------:|
|val ds = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/path/file.csv") |loads the content of file.csv into a dataset ds by indicting that it contains a header  and by requesting  Spark to infer the schema | 
|ds.printSchema | show the schema of ds |
|ds.show(truncate=false)|shows the first 20 rows without truncating the values |
|ds.describe().show()|collects and shows descriptive statistics (mean, max, count, ..) of numeric values|
|ds.select("c1", "c2", ..., "cn")|projects ds on the columns c1, …, cn|
|ds.withColumnRenamed("c1","c2")|renames the column c1 with c2|
|ds.where(cond)|selects the rows respecting cond|
|ds.groupBy("c1").agg(collect_list($"c2") as "values")|groups the rows by column c1 and creates an new column of values associated to those of c1|
|ds.groupBy("c1").agg(avg("c2"))|computes the sum of c2 for each c1 |
|ds.withColumn("new", Exp)|creates a new column whose values are computed by Exp|
|ds1.crossJoin(ds2)|computes the cross product of ds1 and ds2|
|ds1.join(ds2, "c") |joins ds1 and ds2 on the column c|
|ds1.join(ds2, Seq("c1",...,"cn")) |generalizes the previous one to a sequence of columns c1,…, cn|

## Questions

### Chargement des données 

In [2]:
val path = "C:/Users/yousef/Desktop/Books/"
//"/tmp/BDLE/dataset/Books/" 
val users_df = spark.read.format("csv").
option("header", "true").option("inferSchema", "true").
load(path +"users.csv")

val books_df = spark.read.format("csv")
.option("header", "true").option("inferSchema", "true")
.load(path +"books.csv")

val ratings_df = spark.read.format("csv")
.option("header", "true").option("inferSchema", "true")
.load(path +"ratings.csv")

path: String = C:/Users/yousef/Desktop/Books/
users_df: org.apache.spark.sql.DataFrame = [userid: int, country: string ... 1 more field]
books_df: org.apache.spark.sql.DataFrame = [bookid: int, titlewords: int ... 3 more fields]
ratings_df: org.apache.spark.sql.DataFrame = [userid: int, bookid: int ... 1 more field]


In [3]:
users_df.printSchema()
books_df.printSchema()
ratings_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- age: integer (nullable = true)

root
 |-- bookid: integer (nullable = true)
 |-- titlewords: integer (nullable = true)
 |-- authorwords: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- publisher: integer (nullable = true)

root
 |-- userid: integer (nullable = true)
 |-- bookid: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [4]:
users_df.describe().show()
books_df.describe().show()
ratings_df.describe().show()

+-------+------------------+-------+------------------+
|summary|            userid|country|               age|
+-------+------------------+-------+------------------+
|  count|             27876|  27876|             27876|
|   mean|139492.66404792652|   null| 23.93001147940881|
| stddev| 80461.72293106595|   null|20.897392248233995|
|    min|                 8|albania|                 0|
|    max|            278854|    ysa|               244|
+-------+------------------+-------+------------------+

+-------+-----------------+------------------+-------------------+------------------+------------------+
|summary|           bookid|        titlewords|        authorwords|              year|         publisher|
+-------+-----------------+------------------+-------------------+------------------+------------------+
|  count|            49972|             49972|              49972|             49972|             49972|
|   mean|69322.26789001841|5.6949891939486115|  2.196490034419275|1959.3767

Formuler les requêtes retournant les infromations suivantes.
Réponsre à chaque question en déclarant une variable portant le même nom que la question.
Exemple: `val s0 = ….` pour la question s0.

Pour visualiser les résultats d'une requête, invoquer la méthode `show()` sur la valeur associée.
Exemple: `s0.show()`


### Requêtes simples

#### s0) Identifiants des utilisateurs du pays 'france'

In [5]:
val s0 = users_df.where("country=\"france\"")
s0.show()


+------+-------+---+
|userid|country|age|
+------+-------+---+
|100681| france| 54|
|101742| france| 26|
|105187| france| 53|
|106040| france| 29|
|107767| france| 17|
|108153| france| 53|
|108551| france| 28|
|109040| france| 26|
|110367| france| 21|
|110572| france| 18|
|113275| france| 38|
|114043| france| 58|
|115198| france| 31|
|115259| france| 32|
|117941| france| 33|
| 11881| france| 32|
|118846| france| 29|
|119859| france| 53|
|120332| france| 41|
|121070| france| 27|
+------+-------+---+
only showing top 20 rows



s0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userid: int, country: string ... 1 more field]


#### s1) Identifiants des livres  publiés en 2000

In [6]:
val s1 = books_df.where("year=2000")
s1.show()

+------+----------+-----------+----+---------+
|bookid|titlewords|authorwords|year|publisher|
+------+----------+-----------+----+---------+
|    29|         1|          2|2000|       28|
|    37|         2|          2|2000|       32|
|    45|         1|          3|2000|       39|
|    52|        12|          2|2000|       42|
|    81|         2|          2|2000|       61|
|    84|         2|          2|2000|       64|
|   102|        19|          3|2000|       74|
|   114|         2|          2|2000|       74|
|   120|         2|          2|2000|       80|
|   127|         6|          2|2000|       40|
|   162|         3|          2|2000|       74|
|   170|         3|          3|2000|       26|
|   202|         5|          2|2000|      126|
|   212|         3|          2|2000|       75|
|   219|         2|          2|2000|      129|
|   231|         2|          2|2000|       57|
|   248|         4|          2|2000|      140|
|   283|         3|          2|2000|       83|
|   319|     

s1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [bookid: int, titlewords: int ... 3 more fields]


#### s2) Identifiants des livres évalués avec une note > 3

In [7]:
val s2 = ratings_df.where("rating>3")
s2.show()

+------+------+------+
|userid|bookid|rating|
+------+------+------+
|276747|  4780|     4|
|276747|  1837|     4|
|276772| 33829|     5|
|276772| 83629|     5|
|276788| 19993|     5|
|276814| 14817|     4|
|276822| 17085|     5|
|276822| 34237|     4|
|276822|178121|     5|
|276822|233882|     4|
|276822|  1028|     4|
|276822| 27517|     5|
|276822| 88357|     4|
|276822| 15603|     5|
|276822| 50712|     5|
|276822|140338|     5|
|276847| 82884|     5|
|276847|  8273|     5|
|276847| 52802|     4|
|276847|  5733|     5|
+------+------+------+
only showing top 20 rows



s2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userid: int, bookid: int ... 1 more field]


### Requêtes avec agrégation 


#### q1) Nombre d'utilisateurs par pays  avec tri décroissant

In [30]:
val q1 = users_df.groupBy("country").count().sort(desc("count"))
//ds.groupBy("c1").agg(avg("c2"))
q1.show()

+--------------+-----+
|       country|count|
+--------------+-----+
|           usa|18935|
|        canada| 2505|
|       germany| 1254|
|       unknown| 1069|
|united kingdom| 1019|
|     australia|  581|
|         spain|  518|
|        france|  309|
|         italy|  211|
|      portugal|  184|
|   switzerland|  176|
|   netherlands|  147|
|   new zealand|  113|
|      malaysia|   99|
|       austria|   97|
|     singapore|   52|
|        brazil|   39|
|       finland|   38|
|       ireland|   36|
|   philippines|   32|
+--------------+-----+
only showing top 20 rows



q1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [country: string, count: bigint]


#### q2) Pays qui a le plus grand nombre d'utilisateurs. Il n y a pas d'ex aequo

In [31]:
val q2 = q1.limit(1)
q2.show()

+-------+-----+
|country|count|
+-------+-----+
|    usa|18935|
+-------+-----+



q2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [country: string, count: bigint]


#### q3) Année avec le plus grand nombre de livres édités. Il n y a pas d'ex aequo

In [34]:
val q3 = books_df.groupBy("year").count().sort(desc("count")).limit(1)
q3.show()

+----+-----+
|year|count|
+----+-----+
|2002| 4529|
+----+-----+



q3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [year: int, count: bigint]


#### q4) Editeurs ayant édité plus de 10 livres en totalité

In [38]:
val q4 = books_df.groupBy("publisher").count().where("count>10").sort(desc("count"))
q4.show()

+---------+-----+
|publisher|count|
+---------+-----+
|       28| 1261|
|       40| 1166|
|      149| 1066|
|        7|  998|
|       56|  964|
|       39|  954|
|       74|  885|
|       35|  827|
|       36|  777|
|       57|  776|
|       25|  633|
|      140|  599|
|      160|  587|
|      213|  548|
|       66|  524|
|       92|  518|
|       41|  477|
|       53|  447|
|      134|  425|
|       34|  416|
+---------+-----+
only showing top 20 rows



q4: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [publisher: int, count: bigint]


#### q5) Editeurs ayant édité plus de 5 livres à chaque année où ils ont édité un livre

In [41]:
val q5 = books_df.
q5.show()

q5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [publisher: int]


#### q6) La note moyenne par livre 

In [43]:
val q6 = ratings_df.
q6.show()

q6: org.apache.spark.sql.DataFrame = [bookid: int, avg(rating): double]


#### q7) La nombre de livres ayant été eu le note 5

In [44]:
val q = ratings_df.

q: Long = 47327


#### q8) L'identifiant de l'utilisateur ayant noté le plus grand nombre de livres. Pas d'ex eaquo

In [46]:
val q8 = ratings_df.
q8.show()

q8: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userid: int, count: bigint]


### Requêtes avec jointures

#### q9) Les éditeurs des livres notés par des utilisateurs vivant en France

In [48]:
val q9 = users_df.
q9.show()

q9: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [publisher: int]


#### q10) Les éditeurs des livres n'ayant pas été notés par des utilisateurs vivant en France

In [50]:
val q10 = books_df.
q10.show()

q10: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [publisher: int]


#### q11) Pour chaque pays, le nombre moyen de livres ayant obtenu une note >  3 attribuée par des utilisateurs de ce pays

In [52]:
val q11 = ratings_df.
q11.show()

q11: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [country: string, avg: double]


#### q12) Pour chaque livre, l'age moyen des utilisateurs l'ayant noté

In [54]:
val q12 = ratings_df.
q12.show()

q12: org.apache.spark.sql.DataFrame = [bookid: int, avg(age): double]


#### q13) Paires d'utilisateurs ayant noté le même nombre de livres

In [56]:
val q13 = 

q13.show()

q13a: org.apache.spark.sql.DataFrame = [userid: int, cnt_books: bigint]
q13: org.apache.spark.sql.DataFrame = [uid: int, userid: int]


### Requêtes avec  fonctions utilisateurs

In [6]:
import org.apache.spark.sql.expressions.UserDefinedFunction
import scala.collection.mutable.WrappedArray

import org.apache.spark.sql.expressions.UserDefinedFunction
import scala.collection.mutable.WrappedArray


Définir deux  fonctions utilisateur `unionBooks` et `commonBooks` qui prennent en entrée deux ensembles d'identifiants de livre. 
* unionBooks retourne la cardinalité de l'union des deux ensembles de entrée
* commonBooks retourne la cardinalité de l'intersection des deux ensembles de entrée

Les ensembles en entrée sont stockés avec le type WrappedArray[Integer] de Scala déjà importé.

Les résultats sont de type Double

In [ ]:
def commonBooks: UserDefinedFunction =  udf((l: WrappedArray[/*a completer*/], r: WrappedArray[/*a completer*/]) => /*a completer*/) 
                                            

In [ ]:
def unionBooks: UserDefinedFunction =  udf((l: WrappedArray[/*a completer*/], r: WrappedArray[/*a completer*/]) => /*a completer*/) 

####  q14) Paires d'utilsateurs avec le nombre de livres en communn qu'ils ont notés

In [39]:
val q14a = rating_df.groupBy("userid").agg(collect_list($"bookid") as "books1").withColumnRenamed("userid","uid1")
val q14b=q14a.crossJoin(q14a.withCoulumnRenamed("uid1","uid2").withColumnRenamed("books1","books2"))
q14b.show()

<console>: 24: error: not found: value rating_df

#### q15) Paires d'utilsateurs avec la similarité Jaccard calculée sur les livres qu'ils ont notés.
(Consulter https://en.wikipedia.org/wiki/Jaccard_index)

Utiliser les fonctions `commonBooks` et `unionBooks`


In [62]:
val q15 = 
q15.show()

q15: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userid_1: int, userid_2: int ... 1 more field]


#### == Fin du TME == 